In [26]:
# 导入tushare
import tushare as ts
import pandas as pd

In [27]:
ahldf = pd.read_csv(r"data\AH_stock_list.csv", encoding="utf-8-sig")
usdcnhdf = pd.read_csv(r"data\USDCNH_daily.csv", encoding="utf-8-sig")
usdhkddf = pd.read_csv(r"data\USDHKD_daily.csv", encoding="utf-8-sig")
ah_a_k = pd.read_csv(r"data\AH_stock_daily_A_k.csv", encoding="utf-8-sig")
ah_a_adj = pd.read_csv(r"data\AH_stock_daily_A_adj_factor.csv", encoding="utf-8-sig")
ah_hk_k = pd.read_csv(r"data\AH_stock_daily_HK_k.csv", encoding="utf-8-sig")
ah_hk_adj = pd.read_csv(r"data\AH_stock_daily_HK_adj_factor.csv", encoding="utf-8-sig")

In [28]:
ah_k = ah_a_k[["ts_code", "trade_date", "close"]]
ah_k = ahdf = pd.merge(ah_k, ah_a_adj, on=["ts_code", "trade_date"], how="inner")
ah_k

,ts_code,trade_date,close,adj_factor
0,000039.SZ,20250704,8.02,50.9116
1,000039.SZ,20250703,8.11,50.9116
2,000039.SZ,20250702,8.30,50.9116
3,000039.SZ,20250701,7.89,50.9116
4,000039.SZ,20250630,7.86,50.9116
...,...,...,...,...
306867,688981.SH,20200722,79.57,1.0000
306868,688981.SH,20200721,78.63,1.0000
306869,688981.SH,20200720,79.17,1.0000
306870,688981.SH,20200717,77.06,1.0000


In [29]:
ah_k = ah_k.sort_values(["ts_code", "trade_date"])
ah_k["adj_base_A"] = ah_k.groupby("ts_code")["adj_factor"].transform("last")
ah_k["p_a_adj"] = ah_k["close"] * (ah_k["adj_factor"] / ah_k["adj_base_A"])
ah_k

,ts_code,trade_date,close,adj_factor,adj_base_A,p_a_adj
3619,000039.SZ,20100617,12.70,20.847,50.9116,5.200326
3618,000039.SZ,20100618,12.35,20.847,50.9116,5.057010
3617,000039.SZ,20100621,12.97,20.847,50.9116,5.310884
3616,000039.SZ,20100622,12.85,20.847,50.9116,5.261747
3615,000039.SZ,20100623,12.62,20.847,50.9116,5.167568
...,...,...,...,...,...,...
305671,688981.SH,20250630,88.17,1.000,1.0000,88.170000
305670,688981.SH,20250701,87.76,1.000,1.0000,87.760000
305669,688981.SH,20250702,86.10,1.000,1.0000,86.100000
305668,688981.SH,20250703,86.24,1.000,1.0000,86.240000


In [30]:
ah_k = ah_k.rename(columns={"close": "p_a"})
ah_k = ah_k.rename(columns={"adj_factor": "adj_factor_A"})
ah_k = ah_k[["ts_code", "trade_date", "p_a_adj", "p_a", "adj_factor_A", "adj_base_A"]]
ah_k = ah_k.rename(columns={"ts_code": "ts_code_A"})
ah_k

,ts_code_A,trade_date,p_a_adj,p_a,adj_factor_A,adj_base_A
3619,000039.SZ,20100617,5.200326,12.70,20.847,50.9116
3618,000039.SZ,20100618,5.057010,12.35,20.847,50.9116
3617,000039.SZ,20100621,5.310884,12.97,20.847,50.9116
3616,000039.SZ,20100622,5.261747,12.85,20.847,50.9116
3615,000039.SZ,20100623,5.167568,12.62,20.847,50.9116
...,...,...,...,...,...,...
305671,688981.SH,20250630,88.170000,88.17,1.000,1.0000
305670,688981.SH,20250701,87.760000,87.76,1.000,1.0000
305669,688981.SH,20250702,86.100000,86.10,1.000,1.0000
305668,688981.SH,20250703,86.240000,86.24,1.000,1.0000


In [31]:
ahldf.columns

Index(['ts_code_A', 'name', 'ts_code_HK'], dtype='object')

In [32]:
ah_map = ahldf[["ts_code_A", "ts_code_HK", "name"]]
ah_k = pd.merge(ah_k, ah_map, on="ts_code_A", how="inner")
ah_hk_k = ah_hk_k.rename(columns={"ts_code": "ts_code_HK"})
ah_hk_k = ah_hk_k[["ts_code_HK", "trade_date", "close"]]
ah_k = pd.merge(ah_k, ah_hk_k, on=["ts_code_HK", "trade_date"], how="inner")
ah_hk_adj = ah_hk_adj.rename(columns={"ts_code": "ts_code_HK"})
ah_hk_adj = ah_hk_adj[["ts_code_HK", "trade_date", "adj_factor"]]
ah_hk_adj = ah_hk_adj.rename(columns={"adj_factor": "adj_factor_HK"})
ah_k = pd.merge(ah_k, ah_hk_adj, on=["ts_code_HK", "trade_date"], how="inner")

ah_k = ah_k.sort_values(["ts_code_HK", "trade_date"])
ah_k["adj_base_HK"] = ah_k.groupby("ts_code_HK")["adj_factor_HK"].transform("last")
ah_k["p_hk_adj"] = ah_k["close"] * (ah_k["adj_factor_HK"] / ah_k["adj_base_HK"])

ah_k = ah_k.rename(columns={"close": "p_hk"})
ah_k = ah_k[
    [
        "ts_code_A",
        "ts_code_HK",
        "trade_date",
        "name",
        "p_a_adj",
        "p_hk_adj",
        "p_hk",
        "p_a",
        "adj_factor_HK",
        "adj_factor_A",
        "adj_base_HK",
        "adj_base_A",
    ]
]
ah_k

,ts_code_A,ts_code_HK,trade_date,name,p_a_adj,p_hk_adj,p_hk,p_a,adj_factor_HK,adj_factor_A,adj_base_HK,adj_base_A
11275,000333.SZ,00300.HK,20240919,美的集团,62.270696,64.25,64.25,65.28,1.0,5.347,1.0,5.6054
11276,000333.SZ,00300.HK,20240920,美的集团,63.014739,64.00,64.00,66.06,1.0,5.347,1.0,5.6054
11277,000333.SZ,00300.HK,20240923,美的集团,64.483748,67.60,67.60,67.60,1.0,5.347,1.0,5.6054
11278,000333.SZ,00300.HK,20240924,美的集团,64.903465,70.15,70.15,68.04,1.0,5.347,1.0,5.6054
11279,000333.SZ,00300.HK,20240925,美的集团,66.878041,70.70,70.70,70.11,1.0,5.347,1.0,5.6054
...,...,...,...,...,...,...,...,...,...,...,...,...
236844,688331.SH,09995.HK,20250627,荣昌生物,61.610000,55.55,55.55,61.61,1.0,1.000,1.0,1.0000
236845,688331.SH,09995.HK,20250630,荣昌生物,60.500000,54.45,54.45,60.50,1.0,1.000,1.0,1.0000
236846,688331.SH,09995.HK,20250702,荣昌生物,62.790000,55.25,55.25,62.79,1.0,1.000,1.0,1.0000
236847,688331.SH,09995.HK,20250703,荣昌生物,64.440000,58.95,58.95,64.44,1.0,1.000,1.0,1.0000


In [33]:
usdcnhdf = usdcnhdf[["trade_date", "bid_close"]]
usdcnhdf = usdcnhdf.rename(columns={"bid_close": "usd2cn"})
usdcnhdf

,trade_date,usd2cn
0,20250703,7.16844
1,20250702,7.16888
2,20250701,7.16073
3,20250630,7.15974
4,20250629,7.15662
...,...,...
4046,20120223,6.29660
4047,20120222,6.29600
4048,20120221,6.29640
4049,20120220,6.29180


In [34]:
usdhkddf = usdhkddf[["trade_date", "bid_close"]]
usdhkddf = usdhkddf.rename(columns={"bid_close": "usd2hk"})
usdhkddf

,trade_date,usd2hk
0,20250703,7.84810
1,20250702,7.84963
2,20250701,7.84944
3,20250630,7.84972
4,20250629,7.84956
...,...,...
4481,20100618,7.78129
4482,20100617,7.78219
4483,20100616,7.79069
4484,20100615,7.78949


In [35]:
hk2cn = pd.merge(usdcnhdf, usdhkddf, on=["trade_date"], how="inner")
hk2cn["hk2cn"] = hk2cn["usd2cn"] / hk2cn["usd2hk"]
hk2cn = hk2cn[["trade_date", "hk2cn"]]
hk2cn

,trade_date,hk2cn
0,20250703,0.913398
1,20250702,0.913276
2,20250701,0.912260
3,20250630,0.912101
4,20250629,0.911722
...,...,...
3969,20120224,0.811738
3970,20120223,0.811915
3971,20120222,0.811973
3972,20120221,0.812088


In [36]:
ah_k = pd.merge(ah_k, hk2cn, on=["trade_date"], how="inner")
ah_k

,ts_code_A,ts_code_HK,trade_date,name,p_a_adj,p_hk_adj,p_hk,p_a,adj_factor_HK,adj_factor_A,adj_base_HK,adj_base_A,hk2cn
0,000333.SZ,00300.HK,20240919,美的集团,62.270696,64.25,64.25,65.28,1.0,5.347,1.0,5.6054,0.903838
1,000333.SZ,00300.HK,20240923,美的集团,64.483748,67.60,67.60,67.60,1.0,5.347,1.0,5.6054,0.900349
2,000333.SZ,00300.HK,20240924,美的集团,64.903465,70.15,70.15,68.04,1.0,5.347,1.0,5.6054,0.903029
3,000333.SZ,00300.HK,20240925,美的集团,66.878041,70.70,70.70,70.11,1.0,5.347,1.0,5.6054,0.896083
4,000333.SZ,00300.HK,20240926,美的集团,70.169001,77.70,77.70,73.56,1.0,5.347,1.0,5.6054,0.898161
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214078,688331.SH,09995.HK,20250625,荣昌生物,74.350000,67.05,67.05,74.35,1.0,1.000,1.0,1.0000,0.912076
214079,688331.SH,09995.HK,20250626,荣昌生物,60.700000,59.20,59.20,60.70,1.0,1.000,1.0,1.0000,0.913421
214080,688331.SH,09995.HK,20250630,荣昌生物,60.500000,54.45,54.45,60.50,1.0,1.000,1.0,1.0000,0.912101
214081,688331.SH,09995.HK,20250702,荣昌生物,62.790000,55.25,55.25,62.79,1.0,1.000,1.0,1.0000,0.913276


In [37]:
ah_k["AH_premium_adj"] = (ah_k["p_a_adj"] - (ah_k["p_hk_adj"] * ah_k["hk2cn"])) / (
    ah_k["p_hk_adj"] * ah_k["hk2cn"]
)
ah_k["AH_premium"] = (ah_k["p_a"] - (ah_k["p_hk"] * ah_k["hk2cn"])) / (
    ah_k["p_hk"] * ah_k["hk2cn"]
)
ah_k

,ts_code_A,ts_code_HK,trade_date,name,p_a_adj,p_hk_adj,p_hk,p_a,adj_factor_HK,adj_factor_A,adj_base_HK,adj_base_A,hk2cn,AH_premium_adj,AH_premium
0,000333.SZ,00300.HK,20240919,美的集团,62.270696,64.25,64.25,65.28,1.0,5.347,1.0,5.6054,0.903838,0.072310,0.124130
1,000333.SZ,00300.HK,20240923,美的集团,64.483748,67.60,67.60,67.60,1.0,5.347,1.0,5.6054,0.900349,0.059480,0.110681
2,000333.SZ,00300.HK,20240924,美的集团,64.903465,70.15,70.15,68.04,1.0,5.347,1.0,5.6054,0.903029,0.024562,0.074075
3,000333.SZ,00300.HK,20240925,美的集团,66.878041,70.70,70.70,70.11,1.0,5.347,1.0,5.6054,0.896083,0.055640,0.106655
4,000333.SZ,00300.HK,20240926,美的集团,70.169001,77.70,77.70,73.56,1.0,5.347,1.0,5.6054,0.898161,0.005473,0.054063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214078,688331.SH,09995.HK,20250625,荣昌生物,74.350000,67.05,67.05,74.35,1.0,1.000,1.0,1.0000,0.912076,0.215769,0.215769
214079,688331.SH,09995.HK,20250626,荣昌生物,60.700000,59.20,59.20,60.70,1.0,1.000,1.0,1.0000,0.913421,0.122525,0.122525
214080,688331.SH,09995.HK,20250630,荣昌生物,60.500000,54.45,54.45,60.50,1.0,1.000,1.0,1.0000,0.912101,0.218188,0.218188
214081,688331.SH,09995.HK,20250702,荣昌生物,62.790000,55.25,55.25,62.79,1.0,1.000,1.0,1.0000,0.913276,0.244389,0.244389


In [38]:
filtered_df = ah_k[ah_k["name"] == "中国人寿"]
filtered_df

,ts_code_A,ts_code_HK,trade_date,name,p_a_adj,p_hk_adj,p_hk,p_a,adj_factor_HK,adj_factor_A,adj_base_HK,adj_base_A,hk2cn,AH_premium_adj,AH_premium
151003,601628.SH,02628.HK,20120220,中国人寿,15.223396,16.625835,23.85,18.63,0.6971,1.0810,1.0,1.3229,0.811463,0.128390,-0.037378
151004,601628.SH,02628.HK,20120221,中国人寿,15.460367,16.556125,23.75,18.92,0.6971,1.0810,1.0,1.3229,0.812088,0.149895,-0.019033
151005,601628.SH,02628.HK,20120222,中国人寿,15.525739,16.939530,24.30,19.00,0.6971,1.0810,1.0,1.3229,0.811973,0.128780,-0.037046
151006,601628.SH,02628.HK,20120223,中国人寿,15.566596,16.974385,24.35,19.05,0.6971,1.0810,1.0,1.3229,0.811915,0.129508,-0.036425
151007,601628.SH,02628.HK,20120224,中国人寿,15.713682,16.869820,24.20,19.23,0.6971,1.0810,1.0,1.3229,0.811738,0.147497,-0.021078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154054,601628.SH,02628.HK,20250625,中国人寿,42.430000,19.900000,19.90,42.43,1.0000,1.3229,1.0,1.3229,0.912076,1.337700,1.337700
154055,601628.SH,02628.HK,20250626,中国人寿,42.100000,19.540000,19.54,42.10,1.0000,1.3229,1.0,1.3229,0.913421,1.358775,1.358775
154056,601628.SH,02628.HK,20250630,中国人寿,41.190000,18.840000,18.84,41.19,1.0000,1.3229,1.0,1.3229,0.912101,1.396999,1.396999
154057,601628.SH,02628.HK,20250702,中国人寿,41.060000,18.460000,18.46,41.06,1.0000,1.3229,1.0,1.3229,0.913276,1.435483,1.435483


In [39]:
filtered_df = ah_k[ah_k["trade_date"] == 20250703]
filtered_df = filtered_df.sort_values(by="AH_premium")
filtered_df

,ts_code_A,ts_code_HK,trade_date,name,p_a_adj,p_hk_adj,p_hk,p_a,adj_factor_HK,adj_factor_A,adj_base_HK,adj_base_A,hk2cn,AH_premium_adj,AH_premium
22872,600050.SH,00762.HK,20250703,中国联通,5.33,9.35,9.35,5.33,1.0,1.8048,1.0,1.8048,0.913398,-0.375898,-0.375898
179011,000951.SZ,03808.HK,20250703,中国重汽,17.95,23.55,23.55,17.95,1.0,5.1487,1.0,5.1487,0.913398,-0.165525,-0.165525
174734,300750.SZ,03750.HK,20250703,宁德时代,262.59,343.60,343.60,262.59,1.0,1.9125,1.0,1.9125,0.913398,-0.163309,-0.163309
187086,600036.SH,03968.HK,20250703,招商银行,46.61,52.50,52.50,46.61,1.0,6.0581,1.0,6.0581,0.913398,-0.028015,-0.028015
139956,603259.SH,02359.HK,20250703,药明康德,70.72,79.00,79.00,70.72,1.0,2.5368,1.0,2.5368,0.913398,-0.019935,-0.019935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129366,601238.SH,02238.HK,20250703,广汽集团,7.52,3.01,3.01,7.52,1.0,1.8057,1.0,1.8057,0.913398,1.735213,1.735213
156860,601727.SH,02727.HK,20250703,上海电气,7.34,2.90,2.90,7.34,1.0,1.1430,1.0,1.1430,0.913398,1.771009,1.771009
62847,600876.SH,01108.HK,20250703,凯盛新能,12.13,4.77,4.77,12.13,1.0,1.4270,1.0,1.4270,0.913398,1.784084,1.784084
82163,688505.SH,01349.HK,20250703,复旦张江,8.94,3.51,3.51,8.94,1.0,1.0324,1.0,1.0324,0.913398,1.788498,1.788498


In [40]:
filtered_df.to_csv(
    "data\AH_premium_analysis_last_day.csv", index=False, encoding="utf-8-sig"
)
ah_k.to_csv("data\AH_premium_analysis.csv", index=False, encoding="utf-8-sig")